<a href="https://colab.research.google.com/github/ekkalaks/CS701_stock_prediction/blob/main/Case_Study_1_Stock_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Function and modules for supervised regression models

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor

Function and modules for data analysis and model evaluation

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression

Funtion and modules for deep learning models

In [3]:
from keras.models import Sequential
from keras.layers import Dense
#from keras.optimizers import gradient_descent_v2 #instead of : SDG
#instead of: from keras.optimizers we uses tensorflow.keras.optimizers
from tensorflow.keras.optimizers import SGD 
from keras.layers import LSTM
from keras.wrappers.scikit_learn import KerasRegressor

Function and modules for time series models

In [4]:
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Function and modules for data preparation and visulization

In [5]:
# pandasm pandas_datareader, numpy and matplotlib
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from matplotlib import pyplot
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from pandas.plotting import scatter_matrix
from statsmodels.graphics.tsaplots import plot_acf
import yfinance as yf
#yf.pdr_override()

Loading the Data

In [14]:
stk_tickers = ['MSFT', 'IBM', 'GOOGL']
ccy_tickers = ['DEXJPUS', 'DEXUSUK']
idx_tickers = ['SP500', 'DJIA', 'VIXCLS']

stk_data = yf.download(stk_tickers, start='2017-01-01')
#stk_data = web.get_data_yahoo(stk_tickers)
ccy_data = web.DataReader(ccy_tickers, 'fred', start='2017-01-01')
idx_data = web.DataReader(idx_tickers, 'fred', start='2017-01-01')

[*********************100%***********************]  3 of 3 completed


In [20]:
stk_data.shape

(1203, 18)

In [21]:
ccy_data.co

AttributeError: ignored

In [18]:
idx_data.shape

(1246, 3)